In [3]:
import os
os.chdir('C:\\Users\\prudi\\Desktop\\Data Sets\\amazon-fine-food-reviews')
import pandas as pd
import numpy as np
import nltk
import spacy
import sqlite3


In [12]:
con = sqlite3.connect('database.sqlite') 
filtered_data = pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score != 3 LIMIT 10""", con)
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [11]:
import en_core_web_sm
nlp = en_core_web_sm.load()

In [14]:
for i in range(len(filtered_data['Text'])):
    doc=nlp(filtered_data['Text'][i])
    print(filtered_data['Text'][i])
    for token in doc:
        

I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.
I PRON nsubj
have VERB aux
bought VERB ROOT
several ADJ dobj
of ADP prep
the DET det
Vitality PROPN nmod
canned VERB amod
dog NOUN compound
food NOUN compound
products NOUN pobj
and CCONJ cc
have VERB aux
found VERB conj
them PRON nsubj
all DET appos
to PART aux
be VERB ccomp
of ADP prep
good ADJ amod
quality NOUN pobj
. PUNCT punct
The DET det
product NOUN nsubj
looks VERB ROOT
more ADJ advmod
like ADP prep
a DET det
stew NOUN pobj
than ADP prep
a DET det
processed VERB amod
meat NOUN pobj
and CCONJ cc
it PRON nsubj
smells VERB conj
better ADV advmod
. PUNCT punct
My DET poss
Labrador PROPN nsubj
is VERB ROOT
finicky ADJ acomp
and CCONJ cc
she PRON nsubj
appreciates VERB conj
this DET det
product NOUN dobj
better ADV advmod
t

in ADP prep
the DET det
expensive ADJ amod
version NOUN pobj
, PUNCT punct
Fralinger PROPN npadvmod
's PART case
. PUNCT punct
  SPACE 
Would VERB aux
highly ADV advmod
recommend VERB ROOT
this DET det
candy NOUN dobj
! PUNCT punct
  SPACE 
I PRON nsubj
served VERB ROOT
it PRON dobj
at ADP prep
a DET det
beach NOUN npadvmod
- PUNCT punct
themed VERB amod
party NOUN pobj
and CCONJ cc
everyone NOUN nsubj
loved VERB conj
it PRON dobj
! PUNCT punct
This taffy is so good.  It is very soft and chewy.  The flavors are amazing.  I would definitely recommend you buying it.  Very satisfying!!
This DET det
taffy NOUN nsubj
is VERB ROOT
so ADV advmod
good ADJ acomp
. PUNCT punct
  SPACE 
It PRON nsubj
is VERB ROOT
very ADV advmod
soft ADJ acomp
and CCONJ cc
chewy ADJ conj
. PUNCT punct
  SPACE 
The DET det
flavors NOUN nsubj
are VERB ROOT
amazing ADJ acomp
. PUNCT punct
  SPACE 
I PRON nsubj
would VERB aux
definitely ADV advmod
recommend VERB ROOT
you PRON nsubj
buying VERB ccomp
it PRON dobj
. PU